<b>ССЫЛКА НА ДАШБОРД:</b><br>
http://redash.public.karpov.courses/public/dashboards/AROgIoiFvylbh8lwtuXgkIV5lBvNXbWWQTEnZQdN?org_slug=default

- Выручка, полученная в этот день.
- Суммарная выручка на текущий день.
- Прирост выручки, полученной в этот день, относительно значения выручки за предыдущий день.

In [ ]:
with date_revenue as(
select date,
       round(sum(price)) as revenue
from products
join (select creation_time::date as date,
              unnest(product_ids) as product_id
      from orders
      where order_id not in (select order_id
                             from user_actions
                             where action = 'cancel_order')) as t1 using(product_id)
group by date
order by date
),

date_total_revenue as(
select date,
       revenue,
       sum(revenue) over(order by date) as total_revenue
from date_revenue
)

select date,
       revenue,
       total_revenue,
       round(revenue :: decimal / lag(revenue, 1) over(ORDER BY date) * 100 - 100, 2) as revenue_change
from date_total_revenue


- Выручка на пользователя (ARPU) за текущий день.
- Выручка на платящего пользователя (ARPPU) за текущий день.
- Выручка с заказа, или средний чек (AOV) за текущий день.

In [ ]:
with total_revenue as(
select date,
       sum(price) as sum_price
from products
join (select creation_time::date as date,
             order_id,
             unnest(product_ids) as product_id
      from orders
      where order_id not in(select order_id from user_actions where action = 'cancel_order')) as t1 using(product_id)
group by date
),

total_users as (
select time::date as date,
       count(distinct user_id) as user_count
from user_actions
group by date
),

paying_users as (
select time::date as date,
       count(distinct user_id) as paying_user_count
from user_actions
where order_id not in (select order_id from user_actions where action = 'cancel_order')
group by date
),

total_orders as (
select creation_time::date as date,
       count(order_id) as order_count
from orders
where order_id not in (select order_id from user_actions where action = 'cancel_order')
group by date
)


select date,
       round(sum_price::decimal / user_count, 2) as arpu,
       round(sum_price::decimal / paying_user_count, 2) as arppu,
       round(sum_price::decimal / order_count, 2) as aov
from total_revenue
join total_users using(date)
join paying_users using(date)
join total_orders using(date)
order by date


- Накопленная выручка на пользователя (Running ARPU).
- Накопленная выручка на платящего пользователя (Running ARPPU).
- Накопленная выручка с заказа, или средний чек (Running AOV).

In [ ]:
with total_revenue as (SELECT date,
                              sum(price) as sum_price
                       FROM  products
                       join (SELECT creation_time::date as date,
                                                    order_id,
                                                    unnest(product_ids) as product_id
                                             FROM orders
                                             WHERE order_id not in (SELECT order_id
                                                                     FROM   user_actions
                                                                     WHERE  action = 'cancel_order')) as t1 using (product_id)
                                            GROUP BY date),

new_users as (SELECT time::date as date,
                    count(distinct user_id) as new_users
              FROM   user_actions
              WHERE  (time::date, user_id) in (SELECT min(time::date),
                                                      user_id
                                               FROM   user_actions
                                               GROUP BY user_id)
              GROUP BY date),
total_users as (SELECT date,
                       sum(new_users) OVER (ORDER BY date) as total_user_count
                FROM  new_users),
paying_users as (SELECT time::date as date,
                        count(distinct user_id) as paying_user_count
                FROM user_actions
                WHERE order_id not in (SELECT order_id
                                              FROM user_actions
                                       WHERE  action = 'cancel_order')
                                       and (user_id, time::date) in (SELECT user_id,
                                                                         min(time::date)
                                                                  FROM  user_actions
                                                                  WHERE  order_id not in (SELECT order_id
                                                                                          FROM   user_actions
                                                                                          WHERE  action = 'cancel_order')
                GROUP BY date),
total_orders as (SELECT creation_time::date as date,
                        count(order_id) as order_count
                FROM  orders
                WHERE order_id not in (SELECT order_id
                                      FROM  user_actions
                                      WHERE  action = 'cancel_order')
                GROUP BY date)
SELECT date,
       round(sum(sum_price) OVER (ORDER BY date) / total_user_count, 2) as running_arpu,
       round(sum(sum_price) OVER (ORDER BY date) / sum(paying_user_count) OVER(ORDER BY date), 2) as running_arppu,
       round(sum(sum_price) OVER (ORDER BY date) / sum(order_count) OVER (ORDER BY date), 2) as running_aov
FROM   total_revenue
join total_users using (date)
join paying_users using (date)
join total_orders using (date)
ORDER BY date;

- Выручкa на пользователя (ARPU).
- Выручкa на платящего пользователя (ARPPU).
- Выручкa на заказ (AOV).

In [ ]:
with dow_revenue as(SELECT to_char(date, 'Day') as weekday,
                           date_part('isodow', date) as weekday_number,
                           sum(price) as revenue
                    FROM(SELECT date,
                                product_id,
                                price FROM(SELECT creation_time::date as date,
                                                                 order_id,
                                                                 unnest(product_ids) as product_id
                    join products using(product_id)
                    WHERE  order_id not in (SELECT order_id
                                            FROM  user_actions
                                            WHERE  action = 'cancel_order')
                    and date between '2022-08-26'
                    and '2022-09-09') as unnest_price
                    GROUP BY weekday, weekday_number),

total_users as(SELECT to_char(time, 'Day') as weekday,
                      date_part('isodow', time) as weekday_number,
                      count(distinct user_id) as user_count
               FROM user_actions
               WHERE  time between '2022-08-26' and '2022-09-09'
              GROUP BY weekday, weekday_number),

paying_users as(SELECT to_char(time, 'Day') as weekday,
                      date_part('isodow', time) as weekday_number,
                      count(distinct user_id) as paying_user_count
                FROM user_actions
                WHERE  order_id not in(SELECT order_id
                                       FROM user_actions
                                       WHERE action = 'cancel_order')
                                       and time between '2022-08-26' and '2022-09-09'
                                       GROUP BY weekday, weekday_number),

total_orders as(SELECT to_char(creation_time, 'Day') as weekday,
                       date_part('isodow', creation_time) as weekday_number,
                       count(order_id) as order_count
                FROM orders
                WHERE order_id not in(SELECT order_id
                FROM user_actions
                WHERE  action = 'cancel_order')
                and creation_time between '2022-08-26' and '2022-09-09'
                GROUP BY weekday, weekday_number)

SELECT dr.weekday,
       dr.weekday_number,
       round(revenue::decimal / user_count, 2) as arpu ,
       round(revenue::decimal / paying_user_count, 2) as arppu,
       round(revenue::decimal / order_count, 2) as aov
FROM dow_revenue as dr
join total_users using(weekday)
join paying_users using(weekday)
join total_orders using(weekday)
ORDER BY weekday_number

- Выручка, полученная в этот день.
- Выручка с заказов новых пользователей, полученная в этот день.
- Доля выручки с заказов новых пользователей в общей выручке, полученная за этот день.
- Доля выручки с заказов остальных пользователей в общей выручке, полученная за этот день.

In [ ]:
with date_revenue as(
SELECT date,
       round(sum(price)) as revenue
FROM products
join (SELECT creation_time::date as date,
             unnest(product_ids) as product_id
      FROM orders
      WHERE order_id not in (SELECT order_id
                             FROM user_actions
                             WHERE  action = 'cancel_order')) as t1 using(product_id)
      GROUP BY date
      ORDER BY date),

date_total_revenue as(
SELECT date,
       revenue,
       sum(revenue) OVER(ORDER BY date) as total_revenue
 FROM date_revenue),

 date_new_revenue as (
SELECT date,
       sum(price) as new_users_revenue
FROM (SELECT date,
             product_id,
             price
FROM (SELECT order_id,
             creation_time :: date as date,
             unnest(product_ids) as product_id
FROM orders) as t1
join products using (product_id)
WHERE  order_id not in (SELECT order_id
                        FROM user_actions
                        WHERE  action = 'cancel_order')
and order_id in (SELECT order_id
                 FROM user_actions
                 WHERE (time :: date, user_id) in (SELECT min(time :: date),
                                                          user_id
                                                   FROM user_actions
                                                   GROUP BY user_id))) as t2
                GROUP BY 1)

SELECT date,
       revenue,
       new_users_revenue,
       round(new_users_revenue::decimal / revenue * 100, 2) as new_users_revenue_share,
       100 - round(new_users_revenue::decimal / revenue * 100,
                   2) as old_users_revenue_share
FROM   date_revenue join date_new_revenue using(date)
ORDER BY date

- Суммарная выручка, полученная от продажи этого товара за весь период.
- Доля выручки от продажи этого товара в общей выручке, полученной за весь период.

In [ ]:
with name_revenue as(
SELECT name as product_name,
               sum(price) as revenue
FROM (SELECT date,
             order_id,
             name,
             price
      FROM(SELECT creation_time :: date as date,
                  order_id,
                  unnest(product_ids) as product_id
           FROM orders) as t1
      join products using(product_id)) as t2
      WHERE order_id not in (SELECT order_id
                             FROM user_actions
                             WHERE  action = 'cancel_order')
      GROUP BY name
),

share_revenue as(SELECT product_name,
                        revenue,
                        round(revenue::decimal / (SELECT sum(revenue) FROM name_revenue) * 100, 2) as share_in_revenue
                  FROM name_revenue
                  GROUP BY product_name, revenue
                  ORDER BY revenue desc
)

SELECT *
FROM share_revenue
WHERE share_in_revenue >= 0.5
UNION
SELECT 'ДРУГОЕ',
       sum(revenue),
       sum(share_in_revenue)
FROM share_revenue
WHERE share_in_revenue < 0.5
ORDER BY revenue desc

- Выручка, полученная в этот день.
- Затраты, образовавшиеся в этот день.
- Сумма НДС с продажи товаров в этот день.
- Валовая прибыль в этот день (выручка за вычетом затрат и НДС).
- Суммарная выручка на текущий день.
- Суммарные затраты на текущий день.
- Суммарный НДС на текущий день.
- Суммарная валовоя прибыль на текущий день.
- Доля валовой прибыли в выручке за этот день (долю п.4 в п.1).
- Доля суммарной валовой прибыли в суммарной выручке на текущий день (долю п.8 в п.5).



In [ ]:
with revenue_table as (
SELECT date,
       sum(price) as revenue
FROM (SELECT date,
             product_id,
             price as price
      FROM (SELECT order_id,
                   creation_time :: date as date,
                   unnest(product_ids) as product_id
            FROM orders) as t1
     join products using (product_id)
     WHERE order_id not in (SELECT order_id
                            FROM user_actions
                            WHERE  action = 'cancel_order')) as t2
GROUP BY date
),

completed_orders_table as (
SELECT creation_time :: date as date,
       count(distinct order_id) as completed_orders
FROM orders
WHERE order_id not in (SELECT order_id
                       FROM user_actions
                       WHERE action = 'cancel_order')
GROUP BY date
),

delivered_orders_table as (
SELECT time :: date as date,
       count(order_id) as delivered_orders
FROM courier_actions
WHERE action = 'deliver_order'
GROUP BY date
),

cool_couriers as (
SELECT date,
count(courier_id) as good_couriers
FROM (SELECT time :: date as date,
             courier_id,
             count(distinct order_id) as orders_per_day
      FROM courier_actions
      WHERE action = 'deliver_order'
      GROUP BY date, courier_id) as t1
WHERE  orders_per_day >= 5
GROUP BY date
),

vat as (
SELECT creation_time :: date as date,
       sum(tax) as tax
FROM (SELECT *
       FROM (SELECT order_id,
                    creation_time,
                    unnest(product_ids) as product_id
              FROM orders) as t1
      join (SELECT product_id,
                   case when name in ('сахар', 'сухарики', 'сушки', 'семечки', 'масло льняное', 'виноград',
                                      'масло оливковое', 'арбуз', 'батон', 'йогурт', 'сливки', 'гречка', 'овсянка', 'макароны', 'баранина', 'апельсины',
                                      'бублики', 'хлеб', 'горох', 'сметана', 'рыба копченая', 'мука', 'шпроты', 'сосиски', 'свинина', 'рис', 'масло кунжутное',
                                      'сгущенка', 'ананас', 'говядина', 'соль', 'рыба вяленая', 'масло подсолнечное', 'яблоки', 'груши', 'лепешка', 'молоко', 'курица',
                                      'лаваш', 'вафли', 'мандарины') then round(price / 110 * 10,
                   else round(price / 120 * 20, 2) end as tax
              FROM products
              ORDER BY (price - price / 120 * 20) desc, product_id asc) as t2 using (product_id)
      WHERE order_id not in (SELECT order_id
                             FROM user_actions
                            WHERE  action = 'cancel_order')) as t3
      GROUP BY date)
),

main_sub as (
SELECT date,
       revenue,
       case when date_part('month', date) = 8 then 120000 + 140 * completed_orders + 150 * delivered_orders + 400 * coalesce(good_couriers, 0)
       else 150000 + 115 * completed_orders + 150 * delivered_orders + 500 * coalesce(good_couriers, 0) end as costs,
       tax
FROM revenue_table
join completed_orders_table using (date)
join delivered_orders_table using (date)
LEFT JOIN cool_couriers using (date)
join vat using (date)
)

SELECT date,
       revenue,
       round(costs :: decimal, 1) as costs,
       round(tax :: decimal, 2) as tax,
       round(revenue - costs :: decimal - tax, 2) as gross_profit,
       round((sum(revenue) OVER(ORDER BY date)) :: decimal, 1) as total_revenue,
       round((sum(costs) OVER(ORDER BY date)) :: decimal, 1) as total_costs,
       round((sum(tax) OVER(ORDER BY date)) :: decimal, 2) as total_tax,
       round((sum(revenue) OVER(ORDER BY date) - sum(costs) OVER(ORDER BY date) - (sum(tax) OVER(ORDER BY date)) :: decimal),
             2) as total_gross_profit,
       round((revenue - costs - tax) / revenue :: decimal * 100,
             2) as gross_profit_ratio,
       round((sum(revenue) OVER(ORDER BY date) - sum(costs) OVER(ORDER BY date) - sum(tax) OVER(ORDER BY date)) / (sum(revenue) OVER(ORDER BY date)) :: decimal * 100,
             2) as total_gross_profit_ratio
FROM main_sub
ORDER BY date